In [3]:
import sys
sys.path.insert(0, '/home/watts/Software/xgboost/python-package') # for xgboost

In [4]:
import re
import numpy as np
import pandas as pd
import feather
import xgboost as xgb
import feather

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

from sklearn.cross_validation import *
from sklearn.grid_search import GridSearchCV

import pickle


/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/watts/anaconda2/envs/aind-dog/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [5]:
def save_classifier(fname, clf):
    # save the classifier
    with open(fname, 'wb') as fid:
        pickle.dump(clf, fid)

def load_classifier(fname):
    # load it again
    with open(fname, 'rb') as fid:
        clf = pickle.load(fid)
        return clf

In [6]:
df_train = np.load('../cache/train_stage2_fe2.npy')

df1 = pd.read_csv('../cache/stage2_labels.csv')
y = df1['y'].values

df_test = np.load('../cache/test_stage2_fe2.npy')
df = pd.read_csv('../cache/stage2_test_id.csv')
pid = df.ID.values

In [7]:
df_train.shape

(3689, 4689)

In [8]:
print(y)

[1 2 2 ..., 6 4 1]


In [9]:
y = y - 1 #fix for zero bound array

In [10]:
print(y.shape)

(3689,)


In [11]:
np.unique(y)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [12]:
wts = np.load('../cache/stage2_train_weights.npy')

In [11]:
# wts_per_class = np.load('../cache/stage2_train_weights_per_class.npy')
# wts_per_class = wts_per_class.tolist()

In [12]:
xgb_model = xgb.XGBClassifier(objective='multi:softprob', sample_weight=wts)http://localhost:8888/notebooks/kaggle-pm/kaggle-pm/notebooks/5_0_stage2_xgb_gridcv.ipynb#
xgbParams = xgb_model.get_xgb_params()
xgbParams['num_class'] = 9
parameters = {
    'learning_rate': [0.03, 0.035, 0.05], #so called `eta` value
    'max_depth': [5,6,10],
    'min_child_weight': [1,5,10],
    'silent': [1],
    'subsample': [0.8, 0.7],
    'colsample_bytree': [0.8, 0.7],
    'missing':[-999],
    'seed': [1337]
}


clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=StratifiedKFold(y, 5),
                   scoring = 'neg_log_loss',
                   verbose=2, refit=True)


# neg_log_loos:-1.336173588372656
# {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 5, 'min_child_weight': 1,
#  'missing': -999, 'seed': 1337, 'silent': 1, 'subsample': 0.8}
# 2.77967
# 1.71664

In [13]:
xgb_model = xgb.XGBClassifier(objective='multi:softprob', sample_weight=wts)
xgbParams = xgb_model.get_xgb_params()
xgbParams['num_class'] = 9
parameters = {
    'learning_rate': [0.03, 0.035, 0.05], #so called `eta` value
    'max_depth': [5,6,10],
    'min_child_weight': [1,5,10],
    'silent': [1],
    'subsample': [0.8, 0.7],
    'colsample_bytree': [0.8, 0.7],
    'missing':[-999],
    'seed': [1337]
}


clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=StratifiedKFold(y, 5),
                   scoring = 'neg_log_loss',
                   verbose=2, refit=True)


In [14]:
clf.fit(df_train, y)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] colsample_bytree=0.8, learning_rate=0.03, max_depth=5, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 
[CV] colsample_bytree=0.8, learning_rate=0.03, max_depth=5, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 
[CV] colsample_bytree=0.8, learning_rate=0.03, max_depth=5, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 
[CV] colsample_bytree=0.8, learning_rate=0.03, max_depth=5, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 
[CV] colsample_bytree=0.8, learning_rate=0.03, max_depth=5, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 
[CV]  colsample_bytree=0.8, learning_rate=0.03, max_depth=5, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 - 6.1min
[CV] colsample_bytree=0.8, learning_rate=0.03, max_depth=5, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.7 
[CV]  colsample_by

[CV]  colsample_bytree=0.8, learning_rate=0.03, max_depth=5, min_child_weight=10, missing=-999, seed=1337, silent=1, subsample=0.7 - 5.8min
[CV] colsample_bytree=0.8, learning_rate=0.03, max_depth=6, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 
[CV]  colsample_bytree=0.8, learning_rate=0.03, max_depth=5, min_child_weight=10, missing=-999, seed=1337, silent=1, subsample=0.7 - 5.9min
[CV] colsample_bytree=0.8, learning_rate=0.03, max_depth=6, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 
[CV]  colsample_bytree=0.8, learning_rate=0.03, max_depth=6, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 - 6.7min
[CV] colsample_bytree=0.8, learning_rate=0.03, max_depth=6, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.7 


[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed: 42.3min


[CV]  colsample_bytree=0.8, learning_rate=0.03, max_depth=6, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 - 6.7min
[CV] colsample_bytree=0.8, learning_rate=0.03, max_depth=6, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.7 
[CV]  colsample_bytree=0.8, learning_rate=0.03, max_depth=6, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 - 6.7min
[CV] colsample_bytree=0.8, learning_rate=0.03, max_depth=6, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.7 
[CV]  colsample_bytree=0.8, learning_rate=0.03, max_depth=6, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 - 6.6min
[CV] colsample_bytree=0.8, learning_rate=0.03, max_depth=6, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.7 
[CV]  colsample_bytree=0.8, learning_rate=0.03, max_depth=6, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 - 6.8min
[CV] colsample_bytree=0.8, learning_rate=0.03, max_dep

[CV] colsample_bytree=0.8, learning_rate=0.03, max_depth=10, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.7 
[CV]  colsample_bytree=0.8, learning_rate=0.03, max_depth=10, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 -10.3min
[CV] colsample_bytree=0.8, learning_rate=0.03, max_depth=10, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.7 
[CV]  colsample_bytree=0.8, learning_rate=0.03, max_depth=10, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 -10.3min
[CV] colsample_bytree=0.8, learning_rate=0.03, max_depth=10, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.7 
[CV]  colsample_bytree=0.8, learning_rate=0.03, max_depth=10, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 -10.2min
[CV] colsample_bytree=0.8, learning_rate=0.03, max_depth=10, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.7 
[CV]  colsample_bytree=0.8, learning_rate=0.03, max_dept

...

[CV]  colsample_bytree=0.7, learning_rate=0.035, max_depth=10, min_child_weight=10, missing=-999, seed=1337, silent=1, subsample=0.7 - 9.5min
[CV] colsample_bytree=0.7, learning_rate=0.05, max_depth=5, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 
[CV]  colsample_bytree=0.7, learning_rate=0.035, max_depth=10, min_child_weight=10, missing=-999, seed=1337, silent=1, subsample=0.7 - 9.3min
[CV] colsample_bytree=0.7, learning_rate=0.05, max_depth=5, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 
[CV]  colsample_bytree=0.7, learning_rate=0.035, max_depth=10, min_child_weight=10, missing=-999, seed=1337, silent=1, subsample=0.7 - 9.4min
[CV] colsample_bytree=0.7, learning_rate=0.05, max_depth=5, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 
[CV]  colsample_bytree=0.7, learning_rate=0.035, max_depth=10, min_child_weight=10, missing=-999, seed=1337, silent=1, subsample=0.7 - 9.7min
[CV] colsample_bytree=0.7, learning_rate=0

[CV] colsample_bytree=0.7, learning_rate=0.05, max_depth=6, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 
[CV]  colsample_bytree=0.7, learning_rate=0.05, max_depth=5, min_child_weight=10, missing=-999, seed=1337, silent=1, subsample=0.7 - 7.2min
[CV] colsample_bytree=0.7, learning_rate=0.05, max_depth=6, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 
[CV]  colsample_bytree=0.7, learning_rate=0.05, max_depth=5, min_child_weight=10, missing=-999, seed=1337, silent=1, subsample=0.7 - 7.0min
[CV] colsample_bytree=0.7, learning_rate=0.05, max_depth=6, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 
[CV]  colsample_bytree=0.7, learning_rate=0.05, max_depth=5, min_child_weight=10, missing=-999, seed=1337, silent=1, subsample=0.7 - 7.2min
[CV] colsample_bytree=0.7, learning_rate=0.05, max_depth=6, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 
[CV]  colsample_bytree=0.7, learning_rate=0.05, max_depth=6,

[CV]  colsample_bytree=0.7, learning_rate=0.05, max_depth=6, min_child_weight=10, missing=-999, seed=1337, silent=1, subsample=0.7 - 7.8min
[CV] colsample_bytree=0.7, learning_rate=0.05, max_depth=10, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 
[CV]  colsample_bytree=0.7, learning_rate=0.05, max_depth=6, min_child_weight=10, missing=-999, seed=1337, silent=1, subsample=0.7 - 7.9min
[CV] colsample_bytree=0.7, learning_rate=0.05, max_depth=10, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 
[CV]  colsample_bytree=0.7, learning_rate=0.05, max_depth=6, min_child_weight=10, missing=-999, seed=1337, silent=1, subsample=0.7 - 8.0min
[CV] colsample_bytree=0.7, learning_rate=0.05, max_depth=10, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 
[CV]  colsample_bytree=0.7, learning_rate=0.05, max_depth=10, min_child_weight=1, missing=-999, seed=1337, silent=1, subsample=0.8 -10.9min
[CV] colsample_bytree=0.7, learning_rate=0.05, 

[Parallel(n_jobs=5)]: Done 540 out of 540 | elapsed: 937.6min finished


GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[0 1 ..., 3 0], n_folds=5, shuffle=False, random_state=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1,
       sample_weight=array([ 4.57251,  6.40763, ...,  3.91212,  4.57251]),
       scale_pos_weight=1, seed=None, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=5,
       param_grid={'learning_rate': [0.03, 0.035, 0.05], 'max_depth': [5, 6, 10], 'min_child_weight': [1, 5, 10], 'silent': [1], 'subsample': [0.8, 0.7], 'colsample_bytree': [0.8, 0.7], 'missing': [-999], 'seed': [1337]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=2)

In [15]:
print(clf.best_score_)
print(clf.best_params_)

# f1_macro
# 0.4293248036851208
# {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 10, 'min_child_weight': 1, 
#  'missing': -999, 'seed': 1337, 'silent': 1, 'subsample': 0.8}
# 2.74021, 1.72588

# neg_log_loos:-1.336173588372656
# {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 5, 'min_child_weight': 1,
#  'missing': -999, 'seed': 1337, 'silent': 1, 'subsample': 0.8}
# 2.77967
# 1.71664

# -1.3876712019506041 (whien using wts, use xgboost from github)
# {'colsample_bytree': 0.7, 'learning_rate': 0.03, 'max_depth': 5, 
#  'min_child_weight': 5, 'missing': -999, 'seed': 1337, 'silent': 1, 'subsample': 0.8}
# private LB: 2.44337, pub LB: 1.69402

# -1.4902400080711418
# {'colsample_bytree': 0.7, 'learning_rate': 0.04, 'max_depth': 6, 'min_child_weight': 1, 
#  'missing': -999, 'seed': 1337, 'silent': 1, 'subsample': 0.7}
# private LB 2.56616
# public LB: 1.71541

# pip xgboost -1.4811547760008215
# {'colsample_bytree': 0.7, 'learning_rate': 0.03, 'max_depth': 5, 'min_child_weight': 5, 'missing': -999, 'seed': 1337, 'silent': 1, 'subsample': 0.8}
# In [14]:

# accuracy
# 0.5538086202222825
# {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 10, 'min_child_weight': 1, 'missing': -999, 'seed': 1337, 'silent': 1, 'subsample': 0.8}
# In [26]:

0.5538086202222825
{'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 10, 'min_child_weight': 1, 'missing': -999, 'seed': 1337, 'silent': 1, 'subsample': 0.8}


In [26]:
# print(clf.best_score_)
# print(clf.best_params_)
#best score -1.42 (  grid with wts prob not used)

In [27]:
# print(clf.best_score_)
# print(clf.best_params_)
# best score: -1.40 (no wts)

In [16]:
best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
# print('Raw AUC score:', score)
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))



colsample_bytree: 0.8
learning_rate: 0.05
max_depth: 10
min_child_weight: 1
missing: -999
seed: 1337
silent: 1
subsample: 0.8


In [17]:
test_probs = clf.predict_proba(df_test)

In [18]:
test_probs.shape

(986, 9)

In [19]:
print(test_probs)

[[ 0.28907114  0.05654833  0.04600402 ...,  0.04970782  0.02760851
   0.02549118]
 [ 0.79918462  0.02901322  0.0163386  ...,  0.04031471  0.01321625
   0.01192694]
 [ 0.06945485  0.08128208  0.0292874  ...,  0.37008429  0.04579915
   0.02356473]
 ..., 
 [ 0.16406983  0.17439699  0.07424797 ...,  0.08226696  0.0428296
   0.03641725]
 [ 0.08678574  0.13382487  0.03449085 ...,  0.21870473  0.02943469
   0.03259961]
 [ 0.31674215  0.26743397  0.03960963 ...,  0.07589783  0.03772981
   0.03045755]]


In [20]:
submission = pd.DataFrame(test_probs, columns=['class'+str(c+1) for c in range(9)])
submission['ID'] = pid
# submission.to_csv('../submissions/subm_xgb_stage2_with_gridcv.csv', index=False)
# scored 1.70423 on the public leaderboard, 2.40243 on the private leaderboard


submission.to_csv('../submissions/subm_xgb_stage2_with_wt_gridcv.csv', index=False)
# scored 1.72781 on the public leaderboard, 2.48738 on the private leaderboard


In [33]:
save_classifier('../cache/xgb_clf.pkl', clf)